## Data Science Bootcamp or Data Analyst Apprenticeship - see Slack Channel

### Pandas ("panel data")

The Pandas package is used to explore data, e.g.,
 - To calculate statistical values (mean, max, min etc.)
 - To clean data (remove outliers, account for missing data etc.)
 - To restructure data into a nicer format
 - To prepare data for futher processing, e.g., for classification, clustering, visualization etc.

Its data manipulation syntax resembles that of Numpy array operations. The Pandas package is already installed as part of the Anaconda installation. If this is not your case, you first need to install it on your computer before switching to the exercises.

### What are pandas - series and dataframes

Below is a short example of a 'Series' and 'DataFrame'. We won't worry too much about creating this from scratch since typically, you will use Pandas for reading in data from a given file.

#### Pandas Series
A series is effectively a column of data.

In [1]:
import pandas as pd   # use the Pandas package and refer to it as 'pd' to save writing out

data_Series = [12, 8, 10, 11]

attendance = pd.Series(data_Series)

print(attendance)

0    12
1     8
2    10
3    11
dtype: int64


We could make the labelling a bit more helpful.

In [2]:
# Add some row labels

attendance_labelled = pd.Series(data_Series, index = ['week_1', 'week_2', 'week_3', 'week_4'])

print(attendance_labelled)

week_1    12
week_2     8
week_3    10
week_4    11
dtype: int64


#### Pandas DataFrame

A DataFrame is simply a collection of multiple Series.

In [3]:
import pandas as pd   

data_df = {'Python' : [12, 8, 10, 11], 'Web101' : [6, 10, 8, 11], 'AI' : [11, 7, 6, 9]}

profiles = pd.DataFrame(data_df, index = ['week_1', 'week_2', 'week_3', 'week_4'])

print(profiles)

        Python  Web101  AI
week_1      12       6  11
week_2       8      10   7
week_3      10       8   6
week_4      11      11   9


Note that if you don't use "print", the DataFrame will be printed as a table.

In [4]:
profiles

,Python,Web101,AI
week_1,12,6,11
week_2,8,10,7
week_3,10,8,6
week_4,11,11,9


### Reading in data

We will use a local database to give you a more realistic picture of what to expect when working on a typical assignment with data manipulation, but also to promote interesting datasets. The database was chosen on the basis that it is quite small, so it is convenient for your first exercises with Pandas.
More interesting databases can be found here: https://opendata.bristol.gov.uk/pages/homepage/

Let us first read the data from the file and see what type it is.

In [5]:
# Note that all the import statements should go at the top of a script
import os
import pandas as pd

input_file = os.path.join(os.path.abspath(''), 'datafiles', 'public-toilets-community.csv')

toilet_data = pd.read_csv(input_file)

print(type(toilet_data))

<class 'pandas.core.frame.DataFrame'>


Since the data is already of type DataFrame, we can easily print it as a table and check what information it contains.

In [6]:
toilet_data

,OBJECTID,Name,Address,Postcode,Ward,Toilet opening hours,Male (no. of),Urinals (no. of),Female (no. of),Unisex (no.of),...,Automatic opening door,Pull cord monitored (Y/N/Times),Radar Key,Baby change,Family Toilet,Automatic Public Convenience,Attended,Full Time Staffing,geo_shape,geo_point_2d
0,52,Tesco - Callington Road,"Callington Road, BS4 5AY",BS4 5AY,Brislington West,"Mon 8am then 24 hours until 10pm on Sats, Sun ...",Yes,0,Yes,0.0,...,N,Yes- during open times,N,Y,N,N,NaN,N,"{""type"": ""Point"", ""coordinates"": [-2.554650233...","51.4324732718, -2.55465023322"
1,29,Sea Walls Public Toilets,Circular Road By Sea Walls viewing Area,BS9 1ZZ,Stoke Bishop,Monday - Saunday 7:30-18:00,2,1,3,NaN,...,N,N,Y,Y,N,N,N,N,"{""type"": ""Point"", ""coordinates"": [-2.635545426...","51.4701425495, -2.63554542692"
2,54,Avonmouth Medical Centre (Pioneer Medical Group),"1 Collins Street, Bristol",BS11 9JJ,Avonmouth & Lawrence Weston,"Monday-Thursday 08:00 - 18:30, Friday 08:00-17:00",0,0,0,2.0,...,N,Y during open times,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.694532865...","51.4987516075, -2.69453286599"
3,2081,Ridingleaze Medical Centre (Pioneer Medical Gr...,"Ridingleaze, Lawrence Weston",BS11 0QE,Avonmouth & Lawrence Weston,Monday- Friday 08:00 - 18:30,0,0,0,1.0,...,N,no,N,N,N,N,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-2.659817532...","51.5004455574, -2.65981753253"
4,57,Barton Hill Settlement,"41-43 Ducie Road, BS5 0AX",BS5 0AX,Lawrence Hill,Monday - Friday 9:00 - 16:00,2,NaN,2,0.0,...,N,Yes- during open time,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.563674974...","51.4562494756, -2.5636749747"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,641,"Ambition Lawrence Weston, Lawrence Weston Yout...","Lawrence Weston Road, Long Cross, Bristol",BS11 0RX,Avonmouth & Lawrence Weston,Monday to Friday 09:00-16:00 with ad-hoc addit...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-2.653822353...","51.5049424918, -2.65382235353"
103,1745,Helios - Sea Mills Surgery,2 Riverleaze,BS9 2HL,Stoke Bishop,Monday-Friday 08:00-18:30,0,0,0,1.0,...,Y,yes during opening hours,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.650086369...","51.4849119244, -2.65008636941"
104,2417,Merrywood Practice,"Knowle Health Park, Downton Road",BS4 1WH,Filwood,08:30-18:30 Monday-Friday,2,0,2,0.0,...,Y,yes,N,Y,N,N,NaN,Y,"{""type"": ""Point"", ""coordinates"": [-2.596459550...","51.4280468206, -2.59645955096"
105,70,Morrison's - Hartcliffe,"Hareclive Rd, Petersons Avenue, Hartcliffe",BS14 0BE,Hartcliffe & Withywood,7am - 10pm,2,3,3,NaN,...,NaN,Y during open hours,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.599634281...","51.405200404, -2.59963428185"


Note that the printed table doesn't contain all rows, but only the first and last 5 out of a total of 107 rows.

Let's say we are only interested in quickly checking the first rows of the dataset. We can use x.head(n) to print the first *n* rows, where *x* is the DataFrame variable (by default, n=5).

In [7]:
toilet_data.head()

,OBJECTID,Name,Address,Postcode,Ward,Toilet opening hours,Male (no. of),Urinals (no. of),Female (no. of),Unisex (no.of),...,Automatic opening door,Pull cord monitored (Y/N/Times),Radar Key,Baby change,Family Toilet,Automatic Public Convenience,Attended,Full Time Staffing,geo_shape,geo_point_2d
0,52,Tesco - Callington Road,"Callington Road, BS4 5AY",BS4 5AY,Brislington West,"Mon 8am then 24 hours until 10pm on Sats, Sun ...",Yes,0,Yes,0.0,...,N,Yes- during open times,N,Y,N,N,NaN,N,"{""type"": ""Point"", ""coordinates"": [-2.554650233...","51.4324732718, -2.55465023322"
1,29,Sea Walls Public Toilets,Circular Road By Sea Walls viewing Area,BS9 1ZZ,Stoke Bishop,Monday - Saunday 7:30-18:00,2,1,3,NaN,...,N,N,Y,Y,N,N,N,N,"{""type"": ""Point"", ""coordinates"": [-2.635545426...","51.4701425495, -2.63554542692"
2,54,Avonmouth Medical Centre (Pioneer Medical Group),"1 Collins Street, Bristol",BS11 9JJ,Avonmouth & Lawrence Weston,"Monday-Thursday 08:00 - 18:30, Friday 08:00-17:00",0,0,0,2.0,...,N,Y during open times,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.694532865...","51.4987516075, -2.69453286599"
3,2081,Ridingleaze Medical Centre (Pioneer Medical Gr...,"Ridingleaze, Lawrence Weston",BS11 0QE,Avonmouth & Lawrence Weston,Monday- Friday 08:00 - 18:30,0,0,0,1.0,...,N,no,N,N,N,N,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-2.659817532...","51.5004455574, -2.65981753253"
4,57,Barton Hill Settlement,"41-43 Ducie Road, BS5 0AX",BS5 0AX,Lawrence Hill,Monday - Friday 9:00 - 16:00,2,NaN,2,0.0,...,N,Yes- during open time,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.563674974...","51.4562494756, -2.5636749747"


In [8]:
toilet_data.head(2)

,OBJECTID,Name,Address,Postcode,Ward,Toilet opening hours,Male (no. of),Urinals (no. of),Female (no. of),Unisex (no.of),...,Automatic opening door,Pull cord monitored (Y/N/Times),Radar Key,Baby change,Family Toilet,Automatic Public Convenience,Attended,Full Time Staffing,geo_shape,geo_point_2d
0,52,Tesco - Callington Road,"Callington Road, BS4 5AY",BS4 5AY,Brislington West,"Mon 8am then 24 hours until 10pm on Sats, Sun ...",Yes,0,Yes,0.0,...,N,Yes- during open times,N,Y,N,N,NaN,N,"{""type"": ""Point"", ""coordinates"": [-2.554650233...","51.4324732718, -2.55465023322"
1,29,Sea Walls Public Toilets,Circular Road By Sea Walls viewing Area,BS9 1ZZ,Stoke Bishop,Monday - Saunday 7:30-18:00,2,1,3,NaN,...,N,N,Y,Y,N,N,N,N,"{""type"": ""Point"", ""coordinates"": [-2.635545426...","51.4701425495, -2.63554542692"


Similarly, let's say we are only interested in quickly checking the last rows of the dataset. We can use x.tail(n) to print the last *n* rows, where *x* is the DataFrame variable (by default, n=5).

In [10]:
toilet_data.tail()

,OBJECTID,Name,Address,Postcode,Ward,Toilet opening hours,Male (no. of),Urinals (no. of),Female (no. of),Unisex (no.of),...,Automatic opening door,Pull cord monitored (Y/N/Times),Radar Key,Baby change,Family Toilet,Automatic Public Convenience,Attended,Full Time Staffing,geo_shape,geo_point_2d
102,641,"Ambition Lawrence Weston, Lawrence Weston Yout...","Lawrence Weston Road, Long Cross, Bristol",BS11 0RX,Avonmouth & Lawrence Weston,Monday to Friday 09:00-16:00 with ad-hoc addit...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-2.653822353...","51.5049424918, -2.65382235353"
103,1745,Helios - Sea Mills Surgery,2 Riverleaze,BS9 2HL,Stoke Bishop,Monday-Friday 08:00-18:30,0,0,0,1.0,...,Y,yes during opening hours,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.650086369...","51.4849119244, -2.65008636941"
104,2417,Merrywood Practice,"Knowle Health Park, Downton Road",BS4 1WH,Filwood,08:30-18:30 Monday-Friday,2,0,2,0.0,...,Y,yes,N,Y,N,N,NaN,Y,"{""type"": ""Point"", ""coordinates"": [-2.596459550...","51.4280468206, -2.59645955096"
105,70,Morrison's - Hartcliffe,"Hareclive Rd, Petersons Avenue, Hartcliffe",BS14 0BE,Hartcliffe & Withywood,7am - 10pm,2,3,3,NaN,...,NaN,Y during open hours,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.599634281...","51.405200404, -2.59963428185"
106,5153,the Life Boat,"25 High St, Bristol",BS11 0DX,Avonmouth & Lawrence Weston,Monday - Saturday 10:00 - 23:00 Sunday 12:00 ...,1,1,1,NaN,...,N,NaN,N,N,N,NaN,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.675368178...","51.4890035948, -2.67536817895"


In [11]:
toilet_data.tail(2)

,OBJECTID,Name,Address,Postcode,Ward,Toilet opening hours,Male (no. of),Urinals (no. of),Female (no. of),Unisex (no.of),...,Automatic opening door,Pull cord monitored (Y/N/Times),Radar Key,Baby change,Family Toilet,Automatic Public Convenience,Attended,Full Time Staffing,geo_shape,geo_point_2d
105,70,Morrison's - Hartcliffe,"Hareclive Rd, Petersons Avenue, Hartcliffe",BS14 0BE,Hartcliffe & Withywood,7am - 10pm,2,3,3,NaN,...,NaN,Y during open hours,N,Y,N,N,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.599634281...","51.405200404, -2.59963428185"
106,5153,the Life Boat,"25 High St, Bristol",BS11 0DX,Avonmouth & Lawrence Weston,Monday - Saturday 10:00 - 23:00 Sunday 12:00 ...,1,1,1,NaN,...,N,NaN,N,N,N,NaN,Y,Y,"{""type"": ""Point"", ""coordinates"": [-2.675368178...","51.4890035948, -2.67536817895"


##### We can see that there is an 'OBJECTID' which (from reading the documentation) I know is a unique identifier for each location.  Hence, it makes sense to use that as our row index - but first we need to check whether there are any duplicates in the column; if so, then it won't work as an index.  

In [12]:
test = toilet_data['OBJECTID'].duplicated().any() # returns True if any duplicates, False otherwise
print(test)

False


##### Good - now we can read in the DataFrame again, but using the first column as an index column:

In [ ]:
toilet_data = pd.read_csv(input_file, index_col = [0]) # ie use first column as the index
toilet_df = pd.DataFrame(toilet_data)
toilet_df.head()

##### That's all there is to it - the data is now in a DataFrame and ready to be examined.  Similarly, can easily read in from JSON or SQL databases (for those who are familiar with them)

### Examining the data

##### Looking at the first few rows wasn't very helpful - too much information so not nicely laid out.  First command that is normally run on a new dataset is .info()

In [ ]:
toilet_df.info()

### Cleaning and manipulating the data

##### Let's assume that we want to look at the spread of publicly available toilets by post-code so we can see which areas need further work to encourage businesses to join the scheme.  We will want to know when the are open and whether there are both female and accessible toilets by postcode.  The steps we will follow are:
 - Remove columns that are clearly not necessary
 - Look more closely at the data to see if there is any clearly wrong or missing - decide what to do about it
 - Have a look at data by postcode
 - Save as a file so that hard work isn't lost!

In [ ]:
# the drop() function can be used to drop a column

df_rd = toilet_df.drop("Ward", axis = 1) # axis = 1 means we are dropping column; use axis = 0 to drop rows
df_rd.info()


##### Good - "ward" is no longer appearing in our new dataframe.  However:
 - we'd really like to remove many columns at the same time, and
 - if we create a new dataframe every time we make a change then we will end up using lots of memory.  It would be better to change the existing dataframe - so long as recreating it isn't a problem in the event of mistakes!!

In [ ]:
# We'll do both of these steps together
# Put column headings in a list, and use inplace =True as an argument

df_rd.drop(["Changing Place", "Name","Address", "Automatic opening door", "Pull cord monitored (Y/N/Times)","Male (no. of)",
              "Urinals (no. of)","Radar Key","Unisex (no.of)","Accessibility info",
              "Baby change", "Family Toilet", "Automatic Public Convenience", "Attended", "Full Time Staffing", 
              "geo_shape", "geo_point_2d"], axis = 1, inplace = True)



In [ ]:
# Let's look at the result
df_rd

##### I've been rather drastic - I might have wanted to look a bit more carefully at some of the columns before deleting them in real life, but it will do as an example.  
#####  At this stage, I think I will save a copy so that if I stop work and pick it up again later, I don't have to re-run all the previous cells: it's not particularly an issue for this example, but it could be if you are looking at the Air Quality dataset which has several sensors sending hourly data for many years!

In [ ]:
# Similar code to reading in the file
import os

output_file = os.path.join(os.path.abspath(''), 'datafiles', 'dataset_acc.csv')
df_rd.to_csv(output_file)

##### So now I'll re-import pandas library, so that if I want to pick up my work at a later stage, then I can start from here.

In [ ]:
import pandas as pd
import os

input_filev1 = os.path.join(os.path.abspath(''), 'datafiles', 'dataset_acc.csv')

acc_data = pd.read_csv(input_filev1, index_col = [0])

acc_data                         

##### Let's have a look at some of the functions that are available in pandas

In [ ]:
acc_data.describe()

#####  top is the most frequent value in that column.  freq is the frequency with which the 'top' value appears (eg, 2 is that most frequent number of female toilets available and there are 19 locations with this number)

##### However, I am a little surprised that more characteristics aren't shown as a result of the describe() function - I'd normally expect to see the mean, 25% percentile, min and max for numeric columns.However, I can see from the .info() output that some of the columns are of type 'object' and this is a sign that they contain mixed types, eg, integer and strings.  Let's have a closer look - first at the female column.

In [ ]:
acc_data.info() # as before

In [ ]:
acc_data['Female (no. of)'].unique() # gives the number of unique values in the column (aka pandas Series)

##### Even better, we can see the frequency with which each of these occur, as follows

In [ ]:
acc_data['Female (no. of)'].value_counts() # gives the frequency of each value

In [ ]:
acc_data.isna().sum() # gives the total instances of either blank, None or NaN

##### So, putting these together I can deduce that for the Female Column:
- there are 107 rows, of which 25 are either blank or NaN, and 87 have values
- there is 1 value each for 'No' and 'Yes'
- the values are actually being held as strings (presumably why the .describe() function didn't return a wealth of information!

##### First we'll get rid of the NaN: we don't know why they are NaN as we don't know about the source of the data I'd probably want to either do more digging or, if that wasn't possible, leave them as NaN.  However, in the interests of showing some functionality of pandas, I'll replace them all with 99.  

In [ ]:
acc_data['Female (no. of)'].fillna(99, inplace = True) # Fills all NaN & blanks with 99.

acc_data # Let's have a look and see if the NaN has gone

##### OK - looks promising.  Let's just double check by looking at the number of NaNs and the frequency with which values appear

In [ ]:
acc_data.isna().sum()

In [ ]:
acc_data['Female (no. of)'].value_counts() 

##### Now let's address the 'Yes' and 'No' and the fact that the apparent integers are actually strings - all in one step.
##### The .map(XYZ) function applies whatever is defined by XYZ to every value in a panda series.  Let's use this to convert all 'yes' and 'no's.  We won't do it inplace this time as it's quite a drastic change to make and it might go wrong and spoil the existing DataFrame, so let's create another column to capture te output.

In [ ]:
def yes_no(x):
    if x == 'Yes':
        x = 1
    elif x == 'No':
        x = 0
    elif type(x) == str:
        x = int(x)
    return x

acc_data['Female'] = acc_data['Female (no. of)'].map(yes_no) # changes yes to 1 and no to 0.  

In [ ]:

acc_data['Female'].value_counts() # now let's check by looking at the frequency

##### Looks good.  Now let's look at the DataFrame and the .describe() function to see if the results are any different.

In [ ]:
acc_data

In [ ]:
acc_data.describe()

##### Better (don't forget that there are 25 values of 99, which is purely illustrative data).

##### So, now we can get rid of the Female (no. of) column, as follows:

In [ ]:
acc_data.drop('Female (no. of)', axis = 1, inplace = True)

##### Now we need to do the same for the information on Accessible toilets.  We already know that there are no blanks, so we just need to see how many  Yes and No's there are

In [ ]:
acc_data['Accessible (no. of)'].value_counts()

In [ ]:
# A few small differences to the previous map() function, but very similar
def yes(x):
    if x.lower() == 'yes':
        x = 1
    elif type(x) == str:
        x = int(x)
    return x

acc_data['Acc'] = acc_data['Accessible (no. of)'].map(yes) # changes yes to 1 and no to 0.  

In [ ]:
acc_data['Acc'].value_counts()

In [ ]:
# Just give the DataFrame a once-over before deleting the redundant column
acc_data

In [ ]:
acc_data.drop('Accessible (no. of)', axis = 1, inplace = True)

In [ ]:
acc_data.describe()

### Grouping

##### Finally, let's look at 'groupby' - the ability to take combinations of rows or columns if it helps with analysis.  For example, a company might have a number of transactions for each customer, and they all need to be added together to look at the total sales distribution by customer
##### In this case we want to see how the toilets are distributed around the city so grouping by Postcode seems sensible - though probably only based on the first 3 or 4 digits.  It will be easier to remove the last 3 digits of the postcode rather than select the first 3 or 4 digits.  Remember slicing of strings?

In [ ]:
def last_three(string):
    return string[:-4] # note that I've removed 4 characters so that the whitespace is removed as well

acc_data['postcode_v1'] = acc_data['Postcode'].map(last_three)
acc_data

In [ ]:
grouped = acc_data.groupby(['postcode_v1']).sum()
grouped

###### So the Female column doesn't look particularly sensible because we used 99 instead of blanks - you can improve on this in the exercises.  You would also want to carry out some sanity checking on the result to make sure that things like count, sum, mean etc are consistent with the original data, ie the code hasn't gone wrong somewhere!